# Understanding Toronto's City Structure

This notebook will be used for the capstone project for the Applied Data Science course in the IBM Coursera track.

# Part 1: Scraping the Data

In [114]:
import pandas as pd
import numpy as np

import json 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

First, We will scrape the data of postal codes of Toronto neighborhoods from wikipedia. We will start by retrieving the table information using the BeautifulSoup package.

In [8]:
import urllib.request
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
#print(soup.prettify())
right_table=soup.find('table', class_='wikitable sortable')

Next, we will initialize empty lists for each column and iterate over the rows to accumulate the data.  Rows with Borough as 'Not assigned' are ignored.  If the Neighborhood is 'Not Assigned', the Neighborhood is listed as the Borough.  

In [9]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3 and cells[1].find(text = True).rstrip() != "Not assigned":
        A.append(cells[0].find(text=True).rstrip())
        B.append(cells[1].find(text=True).rstrip()) 
        if cells[2].find(text = True).rstrip() == 'Not assigned':
            C.append(cells[1].find(text=True).rstrip())
        else:
            C.append(cells[2].find(text=True).rstrip()) 


Finally, the data is read into a pandas dataframe and cleaned.  The '/' is replaced with a ',' in the Neighborhood column.

In [12]:
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C

df['Neighborhood'] = df['Neighborhood'].str.replace(' / ',', ')
print(df)

    PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
..         ...               ...   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4          Queen's Park, Ontario Provincial Government  
..                                                 ...  
98      The Kingsway, Montgomery Road , Old Mill North  
99                                Church and Wellesley  
100              Business reply mail Processing CentrE

The shape/dimensions are shown below

In [13]:
df.shape

(103, 3)

# Part II: Adding in Latitude and Longitude information via geocoder

Next, we will download the latitude and longitude coordinates for the Neighborhood using the geocoder package.

In [17]:
pip install geocoder


Note: you may need to restart the kernel to use updated packages.


In [169]:
import geocoder # import geocoder

latitude = []
longitude = []
# loop until you get the coordinates
for postal_code in df['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
    latitude.append(g.latlng[0])
    longitude.append(g.latlng[1])

    

Now, append the latitude and longitude columns onto the dataframe with Neighborhoods.

In [50]:
df['Latitude']=latitude
df['Longitude']=longitude
print(df)

    PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
..         ...               ...   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.752935 -79.335641  
1                                     Victoria Village  43.728102 -79.311890  
2                            Regent Park, Harbourfront  43.650964 -79.353041  
3                     Lawrence Manor, Lawrence Heights  43.723265 -79.451211  
4          Queen's Park, Ontario Provincial Government  43.661790 -79.389390  
..                                                 ...        ...        ...  
98      The Ki

In [54]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [61]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: ...working... done

# All requested packages already installed.



After installing folium and finding the coordinates of Toronto, a preliminary map of Toronto with Neighborhoods is shown.

In [62]:

import folium
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Part III: Pulling foursquare API information and Clustering the neighborhoods

We will grab data from the Foursquare API to find distinguishing characteristics of each neighborhood.

In [77]:
CLIENT_ID = 'P3LFOI0HUBKFQDALQZ2VOV3NIZLPCWKMWROPMYMLPMANEO0J' # your Foursquare ID
CLIENT_SECRET = 'QO0JSK5QW42O2K3JJBSXZRDSBEGHPC5AR20EIO5D40RWCE0F' 
VERSION = '20180605'
LIMIT = 100
radius = 500

In [78]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [80]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752935,-79.335641,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
2,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.728102,-79.311890,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [97]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
cols = list(toronto_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighborhood')))

toronto_onehot = toronto_onehot[cols]


toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Find out what percentage of the venues is in each neighborhood radius via groupby.

In [111]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [143]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [162]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Skating Rink,Badminton Court,Supermarket,Sushi Restaurant,Fish Market,Flower Shop,Food,Fish & Chips Shop,Eastern European Restaurant
1,"Alderwood, Long Branch",Gym,Coffee Shop,Pizza Place,Convenience Store,Pharmacy,Gas Station,Pub,Athletics & Sports,Sandwich Place,Dance Studio
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Deli / Bodega,Gas Station,Restaurant,Frozen Yogurt Shop,Supermarket,Fried Chicken Joint,Diner
3,Bayview Village,Construction & Landscaping,Trail,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Donut Shop
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Thai Restaurant,Sandwich Place,Pharmacy,Sushi Restaurant,Juice Bar,Café,Sports Club,Pub


KMeans clustering is run on the neighborhood to find similar neighborhoods based on venues. After some preliminary analysis, 3 clusters appears to be sufficient to capture different regions of the city.

In [163]:
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2])

In [164]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.dtypes
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,0.0,Construction & Landscaping,Food & Drink Shop,Park,Dumpling Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.728102,-79.311890,2.0,Intersection,Portuguese Restaurant,Pizza Place,French Restaurant,Coffee Shop,Park,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,2.0,Pub,Café,Coffee Shop,Athletics & Sports,Bank,French Restaurant,Intersection,Bakery,Tech Startup,Thai Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,2.0,Clothing Store,Cosmetics Shop,Bookstore,Furniture / Home Store,Men's Store,Women's Store,Toy / Game Store,Restaurant,American Restaurant,Coffee Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,2.0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Sandwich Place,Bookstore,Fried Chicken Joint,Spa,Burger Joint,Burrito Place


Now, we can visualize the clusters.

In [165]:

toronto_merged.dropna(inplace = True)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine the clusters

## Cluster 0: Parks & Fields

In [166]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Construction & Landscaping,Food & Drink Shop,Park,Dumpling Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
5,Etobicoke,0.0,Park,Skating Rink,Baseball Field,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Yoga Studio,Dumpling Restaurant
16,York,0.0,Hockey Arena,Field,Grocery Store,Park,Trail,Fish & Chips Shop,Fish Market,Fast Food Restaurant,Farmers Market,Donut Shop
21,York,0.0,Park,Mexican Restaurant,Bakery,Beer Store,Gym,Sporting Goods Shop,Women's Store,Fish Market,Fish & Chips Shop,Field
27,North York,0.0,Park,Residential Building (Apartment / Condo),Dog Run,Yoga Studio,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Farm
41,East Toronto,0.0,Park,Business Service,Grocery Store,Bus Line,Discount Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
49,North York,0.0,Bakery,Park,Basketball Court,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Dumpling Restaurant
57,North York,0.0,Playground,Park,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Fast Food Restaurant
66,North York,0.0,Convenience Store,Park,Bank,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Fast Food Restaurant
67,Central Toronto,0.0,Breakfast Spot,Park,Department Store,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field,Dumpling Restaurant


## Cluster 1: Trails, Construction, and Farms

In [167]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,1.0,Trail,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Donut Shop
26,Scarborough,1.0,Construction & Landscaping,Trail,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Donut Shop
39,North York,1.0,Construction & Landscaping,Trail,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Donut Shop


## Cluster 2: Downtown Core

In [168]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2.0,Intersection,Portuguese Restaurant,Pizza Place,French Restaurant,Coffee Shop,Park,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,Downtown Toronto,2.0,Pub,Café,Coffee Shop,Athletics & Sports,Bank,French Restaurant,Intersection,Bakery,Tech Startup,Thai Restaurant
3,North York,2.0,Clothing Store,Cosmetics Shop,Bookstore,Furniture / Home Store,Men's Store,Women's Store,Toy / Game Store,Restaurant,American Restaurant,Coffee Shop
4,Downtown Toronto,2.0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Sandwich Place,Bookstore,Fried Chicken Joint,Spa,Burger Joint,Burrito Place
7,North York,2.0,Athletics & Sports,Trail,Burger Joint,Bank,Restaurant,Park,Other Great Outdoors,Coffee Shop,Yoga Studio,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,2.0,Coffee Shop,Restaurant,Bakery,Café,Pub,Pizza Place,Italian Restaurant,Gastropub,Beer Store,Taiwanese Restaurant
97,Downtown Toronto,2.0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Japanese Restaurant,Gym,Asian Restaurant,Deli / Bodega,Steakhouse
99,Downtown Toronto,2.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Pub,Dance Studio,Mediterranean Restaurant,Men's Store,Gay Bar,Gastropub
100,East Toronto,2.0,Coffee Shop,Hotel,Café,Japanese Restaurant,American Restaurant,Asian Restaurant,Restaurant,Theater,Italian Restaurant,Sushi Restaurant
